# Process from webcam

In [ ]:
import cv2
from deepface import DeepFace
import FaceDlibs
import numpy as np
import math

# Captura de video desde la cámara
vid = cv2.VideoCapture(0)

# Interfaz de detectores de cara
FDet = FaceDlibs.FaceDlib()

# Parámetros de texto
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (255, 0, 0) 
thickness = 2 
pos = (50, 50)

# Cargamos la imagen con el canal alfa
filter_image = cv2.imread('emojis/gota.png', -1)
star_img = cv2.imread('emojis/happy.png', -1)
barba_img = cv2.imread('emojis/barba.png', -1)


while(True):      
    ret, frame = vid.read()
    B, G, R = cv2.split(frame)

    if ret:
        # Analizamos la imagen para obtener los atributos de la cara con Deepface
        obj = DeepFace.analyze(img_path = frame, enforce_detection=False, actions=['gender', 'emotion'])
        print(obj)
        if len(obj) == 0:
            break
        
        # Carácteristicas y valores de la cara capturada
        face = obj[0]
        #for (x, y) in shape:
        #    cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)
        
        if face['dominant_emotion'] == "sad":
            # Detectamos los ojos
            values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[0], FDet.EyeDetectors[0])

            if values is not None:
                # Valores de la posición de la cara, ojos y forma
                face, eyes, shape = values
                # Coordenadas y dimensión de la ROI de la cara
                [x, y, w, h] = face
                if x > -1:
                    # Coordenadas y dimensión de los ojos (lex: left eye X, ley: left eye Y, rex: right eye X, rey: right eye Y)
                    [lex, ley, rex, rey] = eyes
                    if lex > -1:
                        # Calcular el ángulo de rotación entre los ojos izquierdo y derecho
                        dx = rex - lex
                        dy = rey - ley
                        angle = math.atan2(dy, dx)  # Ángulo en radianes
                        angle_degrees = math.degrees(angle)  # Convertir a grados

                        # Redimensionamos el filtro
                        filter_resized = cv2.resize(filter_image, (40, 40))

                        # Crear una matriz de rotación para ajustarse a la rotación
                        center = (filter_resized.shape[1] // 2, filter_resized.shape[0] // 2)
                        rotation_matrix = cv2.getRotationMatrix2D(center, angle_degrees, 1)
                        rotated_filter = cv2.warpAffine(filter_resized, rotation_matrix, 
                                                        (filter_resized.shape[1], filter_resized.shape[0]))

                        
                        # Colocar el filtro rotado sobre el ojo izquierdo (punto 39 de los puntos carácteristicos de la cara de DLIB)
                        x, y = shape[39]
                        x = x - 40
                        y = y + 10
                        for i in range(rotated_filter.shape[0]):
                            for j in range(rotated_filter.shape[1]):
                                if rotated_filter[i, j][3] > 0:  # Si la transparencia es mayor que 0
                                    frame[y + i, x + j] = rotated_filter[i, j][0:3]

                        # Posicionar el filtro sobre el ojo derecho (punto 42)
                        x, y = shape[42]
                        x = x + 10
                        y = y + 10
                        for i in range(rotated_filter.shape[0]):
                            for j in range(rotated_filter.shape[1]):
                                if rotated_filter[i, j][3] > 0:  # Si la transparencia es mayor que 0
                                    frame[y + i, x + j] = rotated_filter[i, j][0:3]

        elif face['dominant_emotion'] == "happy":
            # Obtener las coordenadas de los ojos
            if face['region']['left_eye'] != None or face['region']['right_eye'] != None:

                left_eye = face['region']['left_eye']
                right_eye = face['region']['right_eye']


                # Redimensionar la imagen del filtro a las dimensiones de los ojos detectados
                star_resize = cv2.resize(star_img, (50, 50))

                # Colocar el filtro sobre el ojo izquierdo (sin considerar transparencia)
                for i in range(star_resize.shape[0]):
                    for j in range(star_resize.shape[1]):
                        if star_resize[i, j][3] > 0:  # Si la transparencia es mayor que 0
                            frame[left_eye[1] + i -25, left_eye[0] + j -25] = star_resize[i, j][0:3]

                # Colocar el filtro sobre el ojo derecho (sin considerar transparencia)
                for i in range(star_resize.shape[0]):
                    for j in range(star_resize.shape[1]):
                        if star_resize[i, j][3] > 0:  # Si la transparencia es mayor que 0
                            frame[right_eye[1] + i -25, right_eye[0] + j-25] = star_resize[i, j][0:3]

        elif face['dominant_emotion'] == "neutral":
            
                # Detectamos los ojos
            values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[0], FDet.EyeDetectors[0])

            if values is not None:
                face, eyes, shape = values

                [x, y, w, h] = face

                """for (x, y) in shape:
                    cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)"""
        
                if x > -1:
                    [lex, ley, rex, rey] = eyes
                    if lex > -1:
                        

                        # Redimensionamos el filtro
                        filter_resized = cv2.resize(barba_img, (480, 480))

                        

                        
                        # Colocar el filtro rotado sobre el ojo izquierdo
                        x_i, y_i = shape[19]
                        
                        """for i in range(filter_resized.shape[0]):
                            for j in range(filter_resized.shape[1]):
                                if filter_resized[i, j][3] > 0:  # Si la transparencia es mayor que 0
                                    frame[y_i + i, x_i + j] = filter_resized[i, j][0:3]

                        # Posicionar el filtro sobre el ojo derecho
                        """
                        x_d, y_d = shape[24]
                        """
                        for i in range(filter_resized.shape[0]):
                            for j in range(filter_resized.shape[1]):
                                if filter_resized[i, j][3] > 0:  # Si la transparencia es mayor que 0
                                    frame[y_d + i, x_d + j] = filter_resized[i, j][0:3]"""

                        x_n, y_n = shape[27]
                        #cv2.circle(frame, (x_n, y_n), 2, (255, 255, 255), -1)

                        x_m=x_n
                        y_m=y_i
                        for i in range(filter_resized.shape[0]):
                            for j in range(filter_resized.shape[1]):
                                if filter_resized[i, j][3] > 0:  # Transparency check
                                    frame[y_m + i - 170, x_m + j - 220] = filter_resized[i, j][0:3]



        
        elif face['dominant_emotion'] == "fear" or face['dominant_emotion'] == "surprise":
            values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[0], FDet.EyeDetectors[0])

            if values is not None:
                face, eyes, shape = values

                [x, y, w, h] = face
                if x > -1:
                    [lex, ley, rex, rey] = eyes
                    if lex > -1:
                        filter_resized = cv2.resize(filter_image, (40, 40))
                        


   
            
                

        # Mostrar el texto con la información de la cara
        image = cv2.putText(frame, str(obj[0]['dominant_gender'] + ' ' + obj[0]['dominant_emotion']), pos, font,
                            fontScale, color, thickness, cv2.LINE_AA) 
        # Mostrar fotograma
        cv2.imshow('Vid', frame)

    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break

# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

Action: emotion: 100%|██████████| 2/2 [00:02<00:00,  1.03s/it]


[{'gender': {'Woman': 25.952187180519104, 'Man': 74.04781579971313}, 'dominant_gender': 'Man', 'region': {'x': 0, 'y': 0, 'w': 639, 'h': 479, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0, 'emotion': {'angry': 0.02195222332375124, 'disgust': 5.74142788867249e-08, 'fear': 0.0020358160327305086, 'happy': 65.80989956855774, 'sad': 0.8803783915936947, 'surprise': 0.37849624641239643, 'neutral': 32.90724158287048}, 'dominant_emotion': 'happy'}]


Action: emotion: 100%|██████████| 2/2 [00:00<00:00,  7.21it/s]


[{'gender': {'Woman': 2.823495678603649, 'Man': 97.17650413513184}, 'dominant_gender': 'Man', 'region': {'x': 0, 'y': 0, 'w': 639, 'h': 479, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0, 'emotion': {'angry': 0.0010544945325818844, 'disgust': 3.3379850668474945e-12, 'fear': 7.546906033439882e-06, 'happy': 25.9466290473938, 'sad': 70.51373720169067, 'surprise': 1.885182954837125e-12, 'neutral': 3.5385772585868835}, 'dominant_emotion': 'sad'}]


Action: emotion: 100%|██████████| 2/2 [00:00<00:00,  8.02it/s]


...

[{'gender': {'Woman': 4.641560837626457, 'Man': 95.35843133926392}, 'dominant_gender': 'Man', 'region': {'x': 0, 'y': 0, 'w': 639, 'h': 479, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0, 'emotion': {'angry': 99.99762773513794, 'disgust': 1.6100660341760857e-14, 'fear': 5.942090375299358e-07, 'happy': 4.435268152747085e-07, 'sad': 0.002363433850405272, 'surprise': 5.824852878272323e-13, 'neutral': 6.168872346279386e-06}, 'dominant_emotion': 'angry'}]


Action: emotion: 100%|██████████| 2/2 [00:00<00:00,  9.20it/s]


[{'gender': {'Woman': 4.627577587962151, 'Man': 95.3724205493927}, 'dominant_gender': 'Man', 'region': {'x': 0, 'y': 0, 'w': 639, 'h': 479, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0, 'emotion': {'angry': 99.7903406495059, 'disgust': 2.305320793797152e-13, 'fear': 0.07147487329728146, 'happy': 8.134526213420313e-05, 'sad': 0.13757367507062407, 'surprise': 3.901502597269711e-13, 'neutral': 0.0005315411588845394}, 'dominant_emotion': 'angry'}]


Action: emotion: 100%|██████████| 2/2 [00:00<00:00,  9.39it/s]


[{'gender': {'Woman': 6.042985990643501, 'Man': 93.95701289176941}, 'dominant_gender': 'Man', 'region': {'x': 0, 'y': 0, 'w': 639, 'h': 479, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0, 'emotion': {'angry': 2.1632996745884223, 'disgust': 1.2258049785064125e-14, 'fear': 2.620952144995244e-05, 'happy': 1.9018338260944635e-06, 'sad': 97.08690625947155, 'surprise': 1.7604788088108703e-14, 'neutral': 0.7497687805306185}, 'dominant_emotion': 'sad'}]
